# Density Matrix Embedding Theory (DMET)

In [21]:
# Installation of tangelo if not already installed.
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop --quiet
    !pip install qulacs pyscf --quiet

## DMET-CCSD on $CaH_2, MgH_2, PbH_2$ <a class="anchor" id="3"></a>

Before we proceed, let's import all the relevant data-structures and classes from `tangelo`:

In [22]:
import json
from tangelo import SecondQuantizedMolecule
from tangelo.problem_decomposition import DMETProblemDecomposition
from tangelo.problem_decomposition.dmet import Localization
from tangelo.algorithms import VQESolver
from tangelo.algorithms import FCISolver

In [77]:
#approximate CaH2 structure by taking the CaH bond lengths (2.003 Angstroms) and assume linear geometry
cah2 = """
Ca   0.0  0.0  0.0
H   0.0  0.0  -2.003
H   0.0  0.0  2.003
"""
mol_cah2 = SecondQuantizedMolecule(cah2, q=0, spin=0, basis="minao")

#approximate MgH2 structure by taking the MgH bond lengths (1.730 Angstroms) and assume linear geometry
mgh2 = """
Mg   0.0  0.0  0.0
H   0.0  0.0  -1.730
H   0.0  0.0  1.730
"""
mol_mgh2 = SecondQuantizedMolecule(mgh2, q=0, spin=0, basis="minao")

# pbh2 = """
# Pb   0.0  0.0  0.0
# H   0.0  0.0  -1.81
# H   0.0  0.0  1.81
# """
# mol_pbh2 = SecondQuantizedMolecule(pbh2, q=0, spin=0, basis="minao", frozen_orbitals=[0, 1, 2, 3, 4])

In [70]:
# Fragment definition = [1, 1, 1] to optimize computation resources, i.e. simulate each atom separately.

def DMET_energy(second_quantized_molecule):
    options_dmet = {"molecule": second_quantized_molecule,
                           "fragment_atoms": [1,1,1],
                           "fragment_solvers": "ccsd",
                           "verbose": True
                           }
    
    dmet = DMETProblemDecomposition(options_dmet)
    dmet.build()
    energy_dmet = dmet.simulate()
    print(f"DMET energy (hartree): \t {energy_dmet}")
    
    ###correlation energy calculation###
    energy_hf = dmet.mean_field.e_tot
    energy_corr = abs(energy_dmet - energy_hf)
    print(f"Correlation energy (hartree): \t {energy_corr}")
    # print(f"Correlation energy (kcal/mol): \t {627.5*energy_corr_butane}")

In [78]:
DMET_energy(mol_mgh2)

 	Iteration =  1
 	----------------
 
	SCF Occupancy Eigenvalues for Fragment Number : # 0
	[1.14672994 2.        ]

	SCF Occupancy Eigenvalues for Fragment Number : # 1
	[0.42663503 2.         2.         2.         2.         2.
 2.        ]

	SCF Occupancy Eigenvalues for Fragment Number : # 2
	[0.42663503 2.         2.         2.         2.         2.
 2.        ]

		Fragment Number : #  1
		------------------------
		Fragment Energy = -200.9194859765619
		Number of Electrons in Fragment = 10.85885109054493
		Fragment Number : #  2
		------------------------
		Fragment Energy = -3.4899414069314623
		Number of Electrons in Fragment = 1.5004602408552907
		Fragment Number : #  3
		------------------------
		Fragment Energy = -3.4899414069314694
		Number of Electrons in Fragment = 1.5004602408552934
 	Iteration =  2
 	----------------
 
	SCF Occupancy Eigenvalues for Fragment Number : # 0
	[1.14672994 2.        ]

	SCF Occupancy Eigenvalues for Fragment Number : # 1
	[0.42663503 2.     

## Comparison with FCI, CCSD

In [ ]:
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop  --quiet
    !pip install pyscf
    !pip install qulacs qiskit qiskit-aer  

In [ ]:
from tangelo import SecondQuantizedMolecule
from tangelo.algorithms import FCISolver, CCSDSolver
from tangelo.algorithms import VQESolver

LiH = [('Li', (0, 0, 0)),('H', (0, 0, 1.5949))] #these triples refer to coordinates
mol_LiH = SecondQuantizedMolecule(LiH, q=0, spin=0, basis="sto-3g")
mol_LiH_t = SecondQuantizedMolecule(LiH, q=0, spin=2, basis="sto-3g")

NH3 = [('N', (0, 0, 0.1)), ('H', (0.94, 0, 0)), ('H', (-0.47, 0.81, 0)), ('H', (-0.47, -0.81, 0))]
mol_NH3 = SecondQuantizedMolecule(NH3, q=0, spin=0, basis="sto-3g")
mol_NH3_t = SecondQuantizedMolecule(NH3, q=0, spin=2, basis="sto-3g")

#Ca->(2.5369,0.155), H -> 2,-0.155), H -> (3.0739,-0.155)
CaH2 = [('Ca', (0, 0.155, 0)), ('H', (-0.5369, -0.155, 0)), ('H', (0.5369, -0.155, 0))]
mol_CaH2 = SecondQuantizedMolecule(CaH2, q=0, spin=0, basis="sto-3g")
mol_CaH2_t = SecondQuantizedMolecule(CaH2, q=0, spin=2, basis="sto-3g")

MgH2 = [('Mg', (0, 0, 0)), ('H', (0, 0, -1.730)), ('H', (0, 0, 1.730))]
mol_MgH2 = SecondQuantizedMolecule(MgH2, q=0, spin=0, basis="sto-3g")
mol_MgH2_t = SecondQuantizedMolecule(MgH2, q=0, spin=2, basis="sto-3g")

In [ ]:
def FCI_CCSD(second_quantized_molecule):
    fci_solver = FCISolver(second_quantized_molecule)
    fci_energy = fci_solver.simulate()
    ccsd_solver = CCSDSolver(second_quantized_molecule)
    ccsd_energy = ccsd_solver.simulate()
    percent_difference = 100*(fci_energy-ccsd_energy)/fci_energy
    print("FCI energy, CCSD energy, percent_difference:")
    return fci_energy, ccsd_energy, percent_difference

In [ ]:
FCI_CCSD(mol_MgH2)

In [7]:
H10="""
H          1.6180339887          0.0000000000          0.0000000000
H          1.3090169944          0.9510565163          0.0000000000
H          0.5000000000          1.5388417686          0.0000000000
H         -0.5000000000          1.5388417686          0.0000000000
H         -1.3090169944          0.9510565163          0.0000000000
H         -1.6180339887          0.0000000000          0.0000000000
H         -1.3090169944         -0.9510565163          0.0000000000
H         -0.5000000000         -1.5388417686          0.0000000000
H          0.5000000000         -1.5388417686          0.0000000000
H          1.3090169944         -0.9510565163          0.0000000000
"""

mol_h10 = SecondQuantizedMolecule(H10, q=0, spin=0, basis="minao")

In the example below, we show resource requirements using standard parameters. Please note that other encodings could reduce the required resources, but resources would still be too much for current hardware.

In [8]:
options_h10_vqe = {"molecule": mol_h10, "qubit_mapping": "jw", "verbose": False}
vqe_h10 = VQESolver(options_h10_vqe)
vqe_h10.build()

Here are some resources estimation that would be needed for a direct VQE calculation on the initial problem, without DMET: for quantum computers in the NISQ era, tackling this head-on is a daunting task.

In [9]:
resources_h10_vqe = vqe_h10.get_resources()
print(resources_h10_vqe)

{'qubit_hamiltonian_terms': 4723, 'circuit_width': 20, 'circuit_depth': 48567, 'circuit_2qubit_gates': 42752, 'circuit_var_gates': 2604, 'vqe_variational_parameters': 350}


### 4.2 DMET-VQE

Here, we demonstrate how to perform  DMET-VQE calculations using Tangelo package. The aim is to obtain improved results (vs HF energy) when compairing to the Full CI method (without using problem decomposition) and also using a quantum algorithm (VQE).

In [10]:
options_h10_dmet = {"molecule": mol_h10,
                    "fragment_atoms": [1]*10,
                    "fragment_solvers": "vqe",
                    "verbose": False
                    }

dmet_h10 = DMETProblemDecomposition(options_h10_dmet)
dmet_h10.build()

The `dmet.build()` method creates fragments (10) from the H10 molecule. When we decompose the ring of atoms into fragments including only one hydrogen atom each, the DMET method creates a fragment orbital (left: the single orbital distribution is shown in both pink and blue, with the colours depicting the phases) and the bath orbital (right: the single orbital distribution of the remaining nine hydrogen atoms is shown in both pink and blue, with the colours depicting the phases). 

<img src="../img/frag_and_bath.png" alt="fragment_and_bath_orbitals" width="450"/>

Resource estimation is done by calling `dmet_h10.get_resources()`. Here, a list of ten dictionaries is returned and stored in `resources_h10_dmet`. Each dictionary refers to a fragment. As every fragment is the same in this system (a single hydrogen atom), we only print one.

In [11]:
resources_h10_dmet = dmet_h10.get_resources()
print(resources_h10_dmet[0])

{'qubit_hamiltonian_terms': 27, 'circuit_width': 4, 'circuit_depth': 100, 'circuit_2qubit_gates': 64, 'circuit_var_gates': 12, 'vqe_variational_parameters': 2}


Compared to a direct VQE algorithm, those resources are greatly reduced: from 20 qubits down to only 4 qubits in our case. Below, `dmet_h10.simulate()` computes the DMET-VQE energy. 

The options currently selected specify that VQE must be run for each fragment, at each iteration of DMET: as such, it may take 2-3 minutes for this cell to finish. The `verbose` option is turned off to hide the lengthy prints: feel free to turn it back on to track the progress of this cell, if you'd like.

In [12]:
dmet_h10.verbose = False
energy_h10_dmet = dmet_h10.simulate()

print(f"DMET energy (hartree): \t {energy_h10_dmet}")

DMET energy (hartree): 	 -5.367523562281509


A comparison with an FCI calculation is then made.

In [13]:
fci_h10 = FCISolver(mol_h10)
energy_h10_fci = fci_h10.simulate()

print(f"FCI energy (hartree): \t {energy_h10_fci}")

FCI energy (hartree): 	 -5.380926000731037


Lastly, we note that the DMET energy is closer to the FCI energy than the HF energy. DMET-VQE results are an improvement but are still not at the FCI level. This discrepancy is attributable to missing three, four ... many body interactions. When dismantling the system into fragments, we get a single hydrogen atom per fragment. Therefore, those fragments cannot propagate higher level (three and more) excitations. DMET user should have in mind this dilemma between fragment sizes and accuracy of the total electronic energy.

In [14]:
energy_h10_hf = dmet_h10.mean_field.e_tot
delta_h10_fci_hf = abs(energy_h10_fci - energy_h10_hf)
delta_h10_fci_dmet = abs(energy_h10_fci - energy_h10_dmet)

print(f"Difference FCI vs HF energies (hartree): \t\t {delta_h10_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (hartree): \t\t {delta_h10_fci_dmet}")
print(f"Difference FCI vs HF energies (kcal/mol): \t\t {627.5*delta_h10_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (kcal/mol): \t {627.5*delta_h10_fci_dmet}")

Difference FCI vs HF energies (hartree): 		 0.11680533556190298
Difference FCI vs DMET-VQE energies (hartree): 		 0.013402438449528375
Difference FCI vs HF energies (kcal/mol): 		 73.29534806509412
Difference FCI vs DMET-VQE energies (kcal/mol): 	 8.410030127079056
